# Part 2: Modeling and Evaluation

In this section, we use the lagged embeddings created in Part 1 (Bag-of-Words, Word2Vec, and GloVe) to build predictive models of brain activity using fMRI voxel data.

We follow these key steps:
1. Load and validate the preprocessed embeddings (X) for each story.
2. Load corresponding fMRI response matrices (Y) for each subject.
3. Fit a ridge regression model for each embedding type to predict Y from X.
4. Compute the mean correlation coefficient (CC) across all voxels to evaluate model performance.
5. Save the trained models and the evaluation metrics for further analysis.

Each embedding is evaluated on the same story to ensure consistency across comparisons.
This part prepares the ground for more advanced evaluation, such as voxel-level analysis and cross-validation.

Before we start, let's load the data and check they are in the right format by checking story timepoint consistency across embeddings

In [1]:
import joblib
import pickle
from pathlib import Path

def load_embeddings(data_dir):
    """Load all three types of embedding data into a dictionary."""
    X_bow = joblib.load(data_dir / "X_lagged_BoW.joblib")
    with open(data_dir / "X_lagged_W2V.pkl", "rb") as f:
        X_w2v = pickle.load(f)
    with open(data_dir / "X_lagged_GloVe.pkl", "rb") as f:
        X_glove = pickle.load(f)
    return {"BoW": X_bow, "Word2Vec": X_w2v, "GloVe": X_glove}

def check_timepoint_consistency(embeddings):
    """Check if timepoint lengths match across BoW, Word2Vec, and GloVe for each story."""
    print("Checking story timepoint consistency across embeddings...\n")
    story_set = set(embeddings["BoW"]) & set(embeddings["Word2Vec"]) & set(embeddings["GloVe"])
    mismatches = []

    for story in sorted(story_set):
        l_bow = embeddings["BoW"][story].shape[0]
        l_w2v = embeddings["Word2Vec"][story].shape[0]
        l_glove = embeddings["GloVe"][story].shape[0]

        if not (l_bow == l_w2v == l_glove):
            mismatches.append((story, l_bow, l_w2v, l_glove))

    if mismatches:
        print("Found mismatches in the following stories:")
        for story, l_b, l_w, l_g in mismatches:
            print(f"- {story}: BoW={l_b}, Word2Vec={l_w}, GloVe={l_g}")
    else:
        print("All stories have consistent timepoint lengths across embeddings.")

# === Run check ===
DATA_DIR = Path("../data")
embeddings = load_embeddings(DATA_DIR)
check_timepoint_consistency(embeddings)

Checking story timepoint consistency across embeddings...

All stories have consistent timepoint lengths across embeddings.


## Regression

In [ ]:
import os
import numpy as np
import pickle
import joblib
from pathlib import Path
import matplotlib.pyplot as plt
import sys

# Append the ridge_utils folder to the module search path so we can import from it
sys.path.append("./ridge_utils")
from ridge import bootstrap_ridge  # Use the bootstrap function defined in ridge.py

# ------------------------------------------------------------------------------
# Define a run_ridge() wrapper function for GPU-accelerated ridge regression.
# This function wraps bootstrap_ridge() to select optimal alpha parameters via cross-validation,
# compute the regression weights, and evaluate the test set performance.
# ------------------------------------------------------------------------------
def run_ridge(X_train, Y_train, X_test, Y_test, alphas=None, cv=15, chunklen=10, nchunks=5):
    """
    Wrapper function for GPU-accelerated ridge regression using bootstrap_ridge.
    Returns a tuple (wt, metrics) where:
      - wt: Regression weights (model) of shape (features, voxels)
      - metrics: A dictionary with keys "mean_cc", "median_cc", "top1_cc",
                 "top5_cc", and "voxel_ccs" containing evaluation metrics.
    
    Parameters
    ----------
    X_train, Y_train : array_like
        Training stimulus and response data.
    X_test, Y_test : array_like
        Test stimulus and response data.
    alphas : array_like or None, optional
        Candidate ridge regularization parameters. If None, defaults to np.logspace(1, 3, 10).
    cv : int, optional
        Number of bootstrap cross-validation runs.
    chunklen : int, optional
        Length of each hold-out chunk (must be tuned to your data).
    nchunks : int, optional
        Number of chunks to hold out per bootstrap run.
    
    Returns
    -------
    wt : array_like, shape (n_features, n_voxels)
        Trained ridge regression weights.
    metrics : dict
        Dictionary of evaluation metrics on the test set.
    """
    if alphas is None:
        alphas = np.logspace(1, 3, 10)

    # Define a simple z-score function
    def zs(v):
        std = v.std(0)
        std[std == 0] = 1
        return (v - v.mean(0)) / std
    
    # Z-score the training and test data
    X_train_z = zs(X_train)
    Y_train_z = zs(Y_train)
    X_test_z  = zs(X_test)
    Y_test_z  = zs(Y_test)
    
    # Run ridge regression with bootstrap cross-validation to determine optimal alphas and weights
    wt, corrs, valphas, _, _ = bootstrap_ridge(
        X_train_z, Y_train_z, X_test_z, Y_test_z,
        alphas=np.array(alphas),
        nboots=cv,
        chunklen=chunklen,
        nchunks=nchunks,
        use_corr=True,
        normalpha=False,
        return_wt=True
    )
    
    # Compute test predictions and evaluate voxel-wise correlations
    Y_pred = np.dot(X_test_z, wt)
    voxel_ccs = np.array([
        np.corrcoef(Y_test_z[:, i], Y_pred[:, i])[0, 1] if np.std(Y_pred[:, i]) > 0 else 0
        for i in range(Y_test_z.shape[1])
    ])
    metrics = {
        "mean_cc": np.mean(voxel_ccs),
        "median_cc": np.median(voxel_ccs),
        "top1_cc": np.percentile(voxel_ccs, 99),
        "top5_cc": np.percentile(voxel_ccs, 95),
        "voxel_ccs": voxel_ccs
    }
    
    return wt, metrics

# ------------------------------------------------------------------------------
# Load precomputed embedding results (from Part 1)
# ------------------------------------------------------------------------------
def load_embeddings(data_dir):
    """
    Load all three types of embedding data into a dictionary.
    
    Parameters
    ----------
    data_dir : Path
        Path to the directory containing Part 1 embedding result files.
    
    Returns
    -------
    embeddings : dict
        Dictionary mapping embedding names ("BoW", "Word2Vec", "GloVe") to their corresponding matrices.
    """
    X_bow = joblib.load(data_dir / "X_lagged_BoW.joblib")
    with open(data_dir / "X_lagged_W2V.pkl", "rb") as f:
        X_w2v = pickle.load(f)
    with open(data_dir / "X_lagged_GloVe.pkl", "rb") as f:
        X_glove = pickle.load(f)
    return {"BoW": X_bow, "Word2Vec": X_w2v, "GloVe": X_glove}

# ------------------------------------------------------------------------------
# Load fMRI data from a subject's directory
# ------------------------------------------------------------------------------
def load_fmri(subject_dir):
    """
    Load fMRI data from .npy files in a subject folder.
    
    Parameters
    ----------
    subject_dir : Path
        Path to the folder containing .npy files for each story.
    
    Returns
    -------
    fmri_data : dict
        Dictionary mapping story names to fMRI response matrices.
    """
    fmri_data = {}
    for file in subject_dir.glob("*.npy"):
        story = file.stem  # Remove file extension
        fmri_data[story] = np.load(file)
    return fmri_data

# ------------------------------------------------------------------------------
# Split stories into training and testing sets (70% / 30%)
# ------------------------------------------------------------------------------
def split_stories(story_list, train_ratio=0.7, random_state=42):
    """
    Split a list of story identifiers into training and testing sets.
    
    Parameters
    ----------
    story_list : list
        List of story identifiers.
    train_ratio : float, optional
        Proportion of stories to use for training.
    random_state : int, optional
        Random seed for reproducibility.
    
    Returns
    -------
    train_stories : list
        List of training story identifiers.
    test_stories : list
        List of testing story identifiers.
    """
    np.random.seed(random_state)
    stories = sorted(story_list)
    np.random.shuffle(stories)
    split_idx = int(len(stories) * train_ratio)
    return stories[:split_idx], stories[split_idx:]

# ------------------------------------------------------------------------------
# Prepare data by concatenating embedding (X) and fMRI (Y) data across stories
# ------------------------------------------------------------------------------
def prepare_data(stories, embedding_dict, fmri_dict):
    """
    Concatenate data across a list of stories.
    
    Parameters
    ----------
    stories : list
        List of story names to include.
    embedding_dict : dict
        Dictionary for one embedding method.
    fmri_dict : dict
        Subject’s fMRI data dictionary.
    
    Returns
    -------
    X : array_like
        Concatenated embedding matrix along the time axis.
    Y : array_like
        Concatenated fMRI response matrix along the time axis.
    """
    X_list, Y_list = [], []
    for story in stories:
        if story in embedding_dict and story in fmri_dict:
            X_list.append(embedding_dict[story])
            Y_list.append(fmri_dict[story])
        else:
            print(f"Warning: Story {story} is missing in embeddings or fMRI data.")
    if len(X_list) == 0 or len(Y_list) == 0:
        raise ValueError("No overlapping stories found!")
    X = np.concatenate(X_list, axis=0)
    Y = np.concatenate(Y_list, axis=0)
    return X, Y

# ------------------------------------------------------------------------------
# Compute voxel-wise correlation coefficient (CC)
# ------------------------------------------------------------------------------
def compute_voxel_cc(Y_true, Y_pred):
    """
    Compute the correlation coefficient for each voxel between true and predicted responses.
    
    Parameters
    ----------
    Y_true, Y_pred : array_like
        Arrays of shape (N_time, V_voxels).
    
    Returns
    -------
    voxel_ccs : array_like
        1D numpy array of correlation coefficients (length V).
    """
    n_voxels = Y_true.shape[1]
    voxel_ccs = []
    for i in range(n_voxels):
        true_voxel = Y_true[:, i]
        pred_voxel = Y_pred[:, i]
        if np.std(true_voxel) == 0 or np.std(pred_voxel) == 0:
            cc = 0.0
        else:
            cc = np.corrcoef(true_voxel, pred_voxel)[0, 1]
        voxel_ccs.append(cc)
    return np.array(voxel_ccs)

# ------------------------------------------------------------------------------
# Train and evaluate GPU-accelerated ridge regression model
# ------------------------------------------------------------------------------
def train_and_evaluate_gpu(X_train, Y_train, X_test, Y_test, subject, emb_name):
    """
    Train a ridge regression model using GPU acceleration.
    
    This function selects the optimal alpha via internal cross-validation
    using run_ridge(), saves the trained model as a .pkl file under the results folder,
    and returns evaluation metrics computed on the test set.
    
    Parameters
    ----------
    X_train, Y_train : array_like
        Training data.
    X_test, Y_test : array_like
        Test data.
    subject : str
        Subject identifier.
    emb_name : str
        Embedding method name (e.g., "BoW").
    
    Returns
    -------
    metrics : dict
        Evaluation metrics including mean CC, median CC, top 1%, and top 5% percentile CC.
    """
    # Define candidate alpha values
    alphas = np.logspace(1, 3, 10)
    # Run the GPU-accelerated ridge regression via run_ridge()
    model, metrics = run_ridge(X_train, Y_train, X_test, Y_test, alphas=alphas, cv=5)
    
    # Save the trained model into the results folder as a .pkl file
    results_dir = Path("results")
    results_dir.mkdir(exist_ok=True)
    model_filename = results_dir / f"ridge_{subject}_{emb_name}.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(model, f)
    print(f"Saved model for {subject} - {emb_name} to {model_filename}")
    print(f"[{subject} - {emb_name}] Evaluation Metrics: {metrics}")
    return metrics

# ------------------------------------------------------------------------------
# Plot the distribution of voxel-wise correlation coefficients (CC)
# ------------------------------------------------------------------------------
def plot_cc_distribution(voxel_ccs, subject, emb_name):
    """
    Plot and save a histogram of the correlation coefficients across voxels.
    
    Parameters
    ----------
    voxel_ccs : array_like
        1D array of correlation coefficients.
    subject : str
        Subject identifier.
    emb_name : str
        Embedding method name.
    """
    plt.figure()
    plt.hist(voxel_ccs, bins=50)
    plt.xlabel("Correlation Coefficient (CC)")
    plt.ylabel("Number of Voxels")
    plt.title(f"CC Distribution for {subject} ({emb_name})")
    plot_filename = f"results/cc_distribution_{subject}_{emb_name}.png"
    plt.savefig(plot_filename)
    plt.close()
    print(f"Saved distribution plot to {plot_filename}")

# ------------------------------------------------------------------------------
# Main pipeline
# ------------------------------------------------------------------------------
def main():
    # Set directories for embedding data (Part 1) and fMRI data for each subject
    data_dir = Path("../data")  # Precomputed embedding files from Part 1
    subject2_dir = Path("../../tmp_ondemand_ocean_mth240012p_symlink/shared/data/subject2")
    subject3_dir = Path("../../tmp_ondemand_ocean_mth240012p_symlink/shared/data/subject3")
    
    # Load embeddings
    embeddings = load_embeddings(data_dir)
    
    # Load fMRI data for each subject
    fmri_subject2 = load_fmri(subject2_dir)
    fmri_subject3 = load_fmri(subject3_dir)
    
    # Identify common stories available in both embeddings and fMRI data
    stories_subject2 = set(embeddings["BoW"].keys()) & set(fmri_subject2.keys())
    stories_subject3 = set(embeddings["BoW"].keys()) & set(fmri_subject3.keys())
    common_stories = sorted(list(stories_subject2 & stories_subject3))
    print(f"Total common stories across embeddings and both subjects: {len(common_stories)}")
    
    # Split the stories into training (70%) and testing (30%) sets
    train_stories, test_stories = split_stories(common_stories, train_ratio=0.7, random_state=42)
    print(f"Training stories: {len(train_stories)}; Testing stories: {len(test_stories)}")
    
    subjects = {"subject2": fmri_subject2, "subject3": fmri_subject3}
    results = {}
    
    # For each subject and for a selected embedding type (here, "BoW"), train and evaluate the model
    for subject, fmri_data in subjects.items():
        results[subject] = {}
        for emb_name in ["BoW"]:   # ["BoW", "Word2Vec", "GloVe"]
            emb_dict = embeddings[emb_name]
            print(f"Processing {subject} with {emb_name} embeddings...")
            X_train, Y_train = prepare_data(train_stories, emb_dict, fmri_data)
            X_test, Y_test = prepare_data(test_stories, emb_dict, fmri_data)
            metrics = train_and_evaluate_gpu(X_train, Y_train, X_test, Y_test, subject, emb_name)
            results[subject][emb_name] = metrics
    
    # Select the best performing embedding based on the average mean CC across both subjects
    best_emb = None
    best_mean_cc = -np.inf
    for emb_name in embeddings.keys():
        if emb_name in results["subject2"] and emb_name in results["subject3"]:
            avg_mean_cc = np.mean([results[subj][emb_name]["mean_cc"] for subj in subjects])
            print(f"Average mean CC for {emb_name}: {avg_mean_cc}")
            if avg_mean_cc > best_mean_cc:
                best_mean_cc = avg_mean_cc
                best_emb = emb_name
    print(f"Best performing embedding overall: {best_emb} with average mean CC {best_mean_cc}")
    
    # For the best embedding, plot the distribution of voxel-wise CC for each subject
    for subject in subjects.keys():
        voxel_ccs = results[subject][best_emb]["voxel_ccs"]
        plot_cc_distribution(voxel_ccs, subject, best_emb)
        
if __name__ == "__main__":
    main()

Total common stories across embeddings and both subjects: 101
Training stories: 70; Testing stories: 31
Processing subject2 with BoW embeddings...


In [ ]:
import json

metrics_filename = results_dir / f"metrics_{subject}_{emb_name}.json"
with open(metrics_filename, "w") as f:
    json.dump({k: (v.tolist() if isinstance(v, np.ndarray) else v) for k, v in metrics.items()}, f, indent=2)

use gpu